readme: this file is used to process raw labeled data (04/08/2019)

In [38]:
import os
os.chdir('/Users/liyuan/desktop/CSAir/codes')
import pandas as pd
import re
import glob
import numpy as np
import time
import jieba
import jieba.posseg as pseg
import jieba.analyse

from process_labeled_data import ProcessLabeledData

In [39]:
pld = ProcessLabeledData()
all_raw_labeled_data = pld.concat_labeled_data('../Source_Data/CSV_files/*.csv')
print('there are %d labeled user reviews' %len(all_raw_labeled_data))
all_raw_labeled_data = pld.encode_labeled_data('../res/all_raw_labeled_data.csv')
all_raw_labeled_data.head()

there are 1551 labeled user reviews


,review,label,label_encoded
0,生日优惠券还有领取时间限制的？！下次希望能标注一下领取时间限制,计划,7
1,参加了11月1号的“11.11购50元抵用券”，至今未收到该优惠券。,计划,7
2,扑街，没有跟星巴克合作5折优惠，写上去干尼玛啊,计划,7
3,请问广州飞墨尔本的航班超额行李要多少钱1kg.我收到你们发的信息是4折，但具体多少钱1kg不知道,计划,7
4,請問帶一個20吋行李箱加電腦後背包是否需加行李費,计划,7


In [40]:
# checking
sents_tok_lst = []
stopwords = [line.strip() for line in open('../Source_Data/stopwords.txt', 'r', encoding='utf-8').readlines()]   
sents = all_raw_labeled_data.review
for sent in sents:
    tokenized_sent = ' '.join(word for word in jieba.cut(sent.strip()) if word not in stopwords)
    # remove digit
    tokenized_sent = re.sub(r'\d+','',tokenized_sent)
    # remove float
    tokenized_sent = re.sub(r"(\d+\.\d+)", '', tokenized_sent)
    tokenized_sent = re.sub(r'[^\w\s]','', tokenized_sent)
    # remove non-chinese characters
    re_words = re.compile(u"[\u4e00-\u9fa5]+")
    res = re.findall(re_words, tokenized_sent)
    if res:
        tokenized_sent = ' '.join([r for r in res])
    sents_tok_lst.append(tokenized_sent)

# reput tokenized reviews back to df
all_raw_labeled_data['review_tokens'] = sents_tok_lst
all_raw_labeled_data = all_raw_labeled_data[['review','review_tokens','label','label_encoded']]
all_raw_labeled_data.head()


,review,review_tokens,label,label_encoded
0,生日优惠券还有领取时间限制的？！下次希望能标注一下领取时间限制,生日 优惠券 领取 时间 下次 希望 标注 领取 时间,计划,7
1,参加了11月1号的“11.11购50元抵用券”，至今未收到该优惠券。,参加 月 号 购 元 抵用券 未 收到 优惠券,计划,7
2,扑街，没有跟星巴克合作5折优惠，写上去干尼玛啊,扑街 星巴克 合作 折 优惠 写上去 干 尼玛,计划,7
3,请问广州飞墨尔本的航班超额行李要多少钱1kg.我收到你们发的信息是4折，但具体多少钱1kg不知道,请问 广州 飞 墨尔本 航班 超额 行李 钱 收到 发 信息 折 钱,计划,7
4,請問帶一個20吋行李箱加電腦後背包是否需加行李費,請 問帶 一個 吋 行李箱 加電腦 後 背包 需加 行李 費,计划,7


In [43]:
# make sure review is typed as "str"
all_raw_labeled_data.review = all_raw_labeled_data.review.astype('str')
all_raw_labeled_data.review_tokens = all_raw_labeled_data.review_tokens.astype('str')

In [44]:
all_raw_labeled_data.to_csv('../res/labeled_data_with_without_tk.csv', index = False)